# Anime Recommendation System

Dataset taken from https://www.kaggle.com/hernan4444/anime-recommendation-database-2020?select=anime.csv

In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
np.random.seed(10)

In [3]:
ratings = pd.read_csv('animelist.csv')
anime_df = pd.read_csv('anime.csv')

In [4]:
ratings_plan = ratings.loc[ratings['watching_status']==6]
ratings_current = ratings.loc[ratings['watching_status']==1]

Watching_status
1: Currently Watching
2: Completed
3: On Hold
4: Dropped
6: Plan to Watch

# Pre-Processing

In [5]:
rand_userIds = np.random.choice(ratings_current['user_id'].unique(), 
                                size=int(len(ratings_current['user_id'].unique())*0.5), 
                                replace=False)
ratings_target = ratings_current.loc[ratings_current['user_id'].isin(rand_userIds)]

In [6]:
print(len(rand_userIds))

151721


In [7]:
#load ratings used for training
ratings_completed = pd.read_csv('rating_complete.csv')
ratings_completed = ratings_completed.loc[ratings_completed['user_id'].isin(rand_userIds)]
ratings_completed.loc[:, 'rating'] = 1

In [8]:
ratings_completed

,user_id,anime_id,rating
0,0,430,1
1,0,1004,1
2,0,3010,1
3,0,570,1
4,0,2762,1
...,...,...,...
57633095,353400,43,1
57633096,353400,18153,1
57633097,353400,15051,1
57633098,353400,32281,1


# Creating Model

In [9]:
# Get a list of all movie IDs
all_animeIds = ratings['anime_id'].unique()

# Placeholders that will hold the training data
users, items, labels = [], [], []

# This is the set of items that each user has interaction with
#user_item_set = set(zip(ratings_completed['user_id'], ratings_completed['anime_id']))

In [4]:
import torch
from torch.utils.data import Dataset

class AnimeLensTrainDataset(Dataset):
    """AnimeLens PyTorch Dataset for Training
    
    Args:
        ratings (pd.DataFrame): Dataframe containing the anime ratings
        all_animeIds (list): List containing all animeIds
    
    """

    def __init__(self, ratings, all_animeIds):
        self.users, self.items, self.labels = self.get_dataset(ratings, all_animeIds)

    def __len__(self):
        return len(self.users)
  
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]

    def get_dataset(self, ratings, all_animeIds):
        users, items, labels = [], [], []
        user_item_set = set(zip(ratings['user_id'], ratings['anime_id']))

        num_negatives = 2
        for u, i in user_item_set:
            users.append(u)
            items.append(i)
            labels.append(1)
            for _ in range(num_negatives):
                negative_item = np.random.choice(all_animeIds)
                while (u, negative_item) in user_item_set:
                    negative_item = np.random.choice(all_animeIds)
                users.append(u)
                items.append(negative_item)
                labels.append(0)

        return torch.tensor(users), torch.tensor(items), torch.tensor(labels)

In [5]:
import torch.nn as nn
from torch.utils.data import DataLoader

class NCF(nn.Module):
    """ Neural Collaborative Filtering (NCF)
    
        Args:
            num_users (int): Number of unique users
            num_items (int): Number of unique items
            ratings (pd.DataFrame): Dataframe containing the anime ratings for training
            all_animeIds (list): List containing all animeIds (train + test)
    """
    
    def __init__(self, num_users, num_items, ratings, all_animeIds):
        super().__init__()
        #embedding of all users, each embedding having 8 dimensions
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=4)
        #embedding of all animes, each embedding having 8 dimensions
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=4)
        self.fc1 = nn.Linear(in_features=8, out_features=32)
        self.fc2 = nn.Linear(in_features=32, out_features=16)
        self.output = nn.Linear(in_features=16, out_features=1)
        self.ratings = ratings
        self.all_animeIds = all_animeIds
        
    def forward(self, user_input, item_input):
        
        # Pass through embedding layers
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)

        # Concat the two embedding layers
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # Pass through dense layer
        vector = nn.ReLU()(self.fc1(vector))
        vector = nn.ReLU()(self.fc2(vector))

        # Output layer
        pred = nn.Sigmoid()(self.output(vector))

        return pred
    
    def training_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        predicted_labels = self(user_input, item_input)
        loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def train_dataloader(self):
        return DataLoader(AnimeLensTrainDataset(self.ratings, self.all_animeIds),
                          batch_size=512)

train_data = DataLoader(animeLensTrainDataset(self.ratings, self.all_animeIds), batch_size=512, num_workers=4)

In [12]:
#init model
num_users = ratings['user_id'].max()+1
num_items = ratings['anime_id'].max()+1
all_animeIds = ratings['anime_id'].unique()
model = NCF(num_users, num_items, ratings_completed, all_animeIds)

In [13]:
#load train_data
train_data = model.train_dataloader()

In [14]:
#training step
optimizer = model.configure_optimizers()

num_epochs = 1
count = 0
for epoch in range(num_epochs):
    batch_idx = 0
    #training loop
    for train_batch in train_data:
        count += 1
        loss = model.training_step(train_batch, batch_idx)
        if count%1000 == 0:
            print("train_loss: ", loss.item())
            print("batch num: ", count)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        batch_idx += 1

train_loss:  0.53106290102005
batch num:  1000
train_loss:  0.39560115337371826
batch num:  2000
train_loss:  0.36299043893814087
batch num:  3000
train_loss:  0.30180659890174866
batch num:  4000
train_loss:  0.30845507979393005
batch num:  5000
train_loss:  0.3465520739555359
batch num:  6000
train_loss:  0.2956867516040802
batch num:  7000
train_loss:  0.35584646463394165
batch num:  8000
train_loss:  0.31699690222740173
batch num:  9000
train_loss:  0.296225368976593
batch num:  10000
train_loss:  0.3023507595062256
batch num:  11000
train_loss:  0.3247435390949249
batch num:  12000
train_loss:  0.29612112045288086
batch num:  13000
train_loss:  0.28672128915786743
batch num:  14000
train_loss:  0.29898232221603394
batch num:  15000
train_loss:  0.2916077971458435
batch num:  16000
train_loss:  0.3008732199668884
batch num:  17000
train_loss:  0.3089844882488251
batch num:  18000
train_loss:  0.32595309615135193
batch num:  19000
train_loss:  0.3123260736465454
batch num:  20000
tr

train_loss:  0.282440721988678
batch num:  163000


# Model Validation

In [15]:
#get a smaller batch of users for testing
test_userIds = np.random.choice(rand_userIds, 
                                size=int(len(rand_userIds)*0.2), 
                                replace=False)

In [16]:
len(test_userIds)

30344

In [17]:
#filter test ratings based on new user batch
ratings_test_users = ratings.loc[ratings['user_id'].isin(test_userIds)]
ratings_target = ratings_target.loc[ratings_target['user_id'].isin(test_userIds)]

In [18]:
#get only 1 review per user for testing
ratings_target['rank_latest'] = ratings_target.groupby(['user_id'])['watching_status'].rank(method='first', ascending=False)
ratings_target = ratings_target.loc[ratings_target['rank_latest']==1]

In [111]:
# User-item pairs for testing
test_user_item_set = set(zip(ratings_target['user_id'], ratings_target['anime_id']))

# Dict of all items that are interacted with by each user
print("A")
ratings_interacted = ratings_test_users.loc[ratings_test_users['watching_status']!=4]
user_interacted_items = ratings_interacted.groupby('user_id')['anime_id'].apply(list).to_dict()
print("B")

hits = []
count = 0
for (u,i) in test_user_item_set:
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(all_animeIds) - set(interacted_items)
    #selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    #test_items = selected_not_interacted + [i]
    test_items = list(not_interacted_items) + [i]
    
    predicted_labels = np.squeeze(model(torch.tensor([u]*len(test_items)), 
                                        torch.tensor(test_items)).detach().numpy())
    #print(predicted_labels)
    
    top10_items = [test_items[i] for i in np.argsort(predicted_labels)[::-1][0:10].tolist()]
    
    if i in top10_items:
        hits.append(1)
    else:
        hits.append(0)
    count += 1
    print(count)
        
print("The Hit Ratio @ 10 is {:.2f}".format(np.average(hits)))

A
B
[8.8257855e-01 8.5482132e-01 8.6875069e-01 ... 4.4892990e-04 3.3392296e-03
 1.2184881e-03]
1
[9.6088326e-01 8.7081140e-01 9.0884000e-01 ... 2.0850528e-05 4.1550393e-05
 6.1485152e-05]
2
[8.7511069e-01 9.0314126e-01 5.0230277e-01 ... 3.4547531e-05 1.4053103e-04
 7.3498154e-01]
3
[9.75045145e-01 8.50741863e-01 9.20044839e-01 ... 1.49514235e-05
 7.77631976e-06 4.70078111e-01]
4
[8.9593071e-01 6.2466687e-01 2.8447559e-01 ... 8.2524464e-05 2.3214545e-04
 6.2477660e-01]
5
[9.5979542e-01 8.5477918e-01 9.0917099e-01 ... 2.1450349e-05 4.8349055e-05
 7.2880626e-01]
6
[9.7060668e-01 9.0343541e-01 9.1664380e-01 ... 5.2838179e-05 4.0899260e-05
 9.7093374e-01]
7
[8.4031773e-01 3.0246946e-01 3.2591406e-02 ... 1.3253994e-05 4.5779520e-06
 9.8679209e-01]
8
[9.0983969e-01 8.7323517e-01 6.9539052e-01 ... 1.8365792e-04 1.3026468e-03
 7.7678585e-01]
9
[9.0277129e-01 9.1232651e-01 5.9775066e-01 ... 8.3941588e-05 1.5902029e-04
 8.4827924e-01]
10
[9.3260741e-01 8.8869452e-01 8.9958417e-01 ... 1.4567688e-0

92
[6.1312294e-01 2.7568963e-01 6.6044223e-01 ... 7.5891519e-05 3.3773592e-04
 8.5095632e-01]
93
[8.4238756e-01 8.5859191e-01 7.1593636e-01 ... 4.3699186e-04 3.4400860e-03
 6.4553612e-01]
94
[9.2905873e-01 8.8414109e-01 8.9542443e-01 ... 1.8678806e-04 5.2907161e-04
 7.0293061e-04]
95
[8.9951587e-01 8.5942233e-01 8.7021643e-01 ... 3.9549224e-04 2.8248685e-03
 9.3789101e-01]
96
[9.5837700e-01 9.0585774e-01 9.1331536e-01 ... 8.8164677e-05 9.4635456e-05
 9.0883058e-01]
97
[9.5687485e-01 7.4496740e-01 9.1335881e-01 ... 8.9482946e-06 1.3517117e-05
 7.7381176e-01]
98
[8.6364782e-01 6.6822267e-01 4.4252297e-01 ... 3.7922274e-04 2.2471191e-03
 5.6636542e-01]
99
[9.5381624e-01 9.0508759e-01 9.1321516e-01 ... 9.4028896e-05 1.5391350e-04
 8.8483882e-01]
100
[9.7265619e-01 9.0222591e-01 9.1824931e-01 ... 4.2724565e-05 3.5537843e-05
 6.0323787e-01]
101
[9.6755612e-01 8.4792441e-01 9.0703911e-01 ... 5.2233783e-05 5.5626995e-05
 7.3945993e-01]
102
[9.6785688e-01 8.2024789e-01 9.2155385e-01 ... 1.24514

 6.4387882e-01]
182
[5.3264654e-01 1.4625749e-01 6.4904577e-01 ... 4.9054375e-05 1.7377116e-04
 6.6421366e-01]
183
[8.6994594e-01 4.1931099e-01 6.8311706e-02 ... 1.9894213e-05 5.4071847e-05
 8.3759838e-01]
184
[8.9774662e-01 5.4298347e-01 1.5154955e-01 ... 4.5918088e-05 1.0181055e-04
 1.6068086e-01]
185
[8.3819622e-01 9.2184848e-01 3.3662173e-01 ... 1.4092077e-05 2.0380747e-05
 9.4502062e-01]
186
[8.4140253e-01 9.0212417e-01 4.3948415e-01 ... 4.2587522e-05 9.9591081e-05
 8.2222277e-01]
187
[8.1183404e-01 9.1252410e-01 2.8570479e-01 ... 1.3857339e-05 2.6416072e-05
 9.8480475e-01]
188
[8.4886456e-01 9.1745794e-01 3.1700909e-01 ... 1.4244244e-05 2.9241630e-05
 6.3781232e-01]
189
[9.1369390e-01 8.3144760e-01 8.4987837e-01 ... 3.1602872e-04 1.8726215e-03
 8.4095603e-01]
190
[9.4616449e-01 8.9268184e-01 9.0140605e-01 ... 7.7531084e-05 3.6126687e-04
 8.7223107e-01]
191
[9.4262069e-01 8.9052856e-01 8.9852232e-01 ... 6.4615859e-05 2.0215585e-04
 7.9058772e-01]
192
[9.6613187e-01 8.5008603e-01 9

 1.85392724e-04 9.16660547e-01]
273
[9.2336220e-01 8.7523174e-01 8.8666070e-01 ... 1.0876905e-04 4.9197598e-04
 6.2777525e-01]
274
[8.9872801e-01 5.6577712e-01 1.5508226e-01 ... 6.1286235e-05 9.8841199e-05
 8.9120513e-01]
275
[8.8037544e-01 9.1771197e-01 3.7628853e-01 ... 1.7659389e-05 4.0298964e-05
 4.0916617e-05]
276
[8.8058168e-01 6.8113756e-01 4.1751593e-01 ... 1.8367034e-04 1.5578542e-03
 9.2216492e-01]
277
[9.1793412e-01 8.7281048e-01 8.8443196e-01 ... 1.3434402e-04 7.1285618e-04
 8.3244598e-01]
278
[9.5780474e-01 8.7938213e-01 9.0275347e-01 ... 2.4859179e-05 6.8976580e-05
 9.5384544e-01]
279
[9.4979692e-01 8.7471092e-01 8.9549750e-01 ... 1.2807816e-04 3.8096562e-04
 8.0593854e-01]
280
[8.9575398e-01 7.1759053e-02 6.8722135e-01 ... 3.0778709e-05 1.9938811e-05
 2.2546819e-05]
281
[8.4260142e-01 9.2072552e-01 3.7203780e-01 ... 2.1432108e-05 2.2318518e-05
 3.6451482e-05]
282
[9.4310033e-01 8.9793199e-01 9.0257937e-01 ... 6.5892847e-05 2.3786786e-04
 2.9458842e-04]
283
[9.7517699e-01

[8.4880239e-01 7.0265758e-01 4.6717504e-01 ... 4.5279707e-04 3.5193977e-03
 7.0378917e-01]
366
[8.9516681e-01 9.0931880e-01 5.3796893e-01 ... 4.3411343e-05 1.6870486e-04
 8.3374906e-01]
367
[9.56073046e-01 8.62672091e-01 9.04319525e-01 ... 4.67970967e-05
 1.15075054e-04 9.16699171e-01]
368
[9.6662539e-01 8.8424569e-01 9.1016948e-01 ... 2.3027802e-05 5.3444979e-05
 3.6413508e-05]
369
[7.4157953e-01 4.5626053e-01 8.0156380e-01 ... 1.7716932e-04 1.0237928e-03
 6.4982766e-01]
370
[8.9624918e-01 6.2674594e-01 2.9179081e-01 ... 8.2170940e-05 3.2302726e-04
 5.4869097e-01]
371
[9.7164977e-01 8.6169726e-01 9.2535615e-01 ... 1.4327457e-05 2.4652534e-05
 1.8975906e-01]
372
[8.3760548e-01 9.1496223e-01 3.2132214e-01 ... 1.3469607e-05 3.1841122e-05
 8.7565213e-01]
373
[9.4797039e-01 8.9879590e-01 9.0499383e-01 ... 1.3810401e-04 1.8019715e-04
 8.7403595e-01]
374
[8.9844459e-01 8.6340499e-01 8.7654960e-01 ... 3.0672259e-04 2.5007259e-03
 8.5327125e-01]
375
[8.7868464e-01 4.4052827e-01 6.3747272e-02 .

...

 2.1191676e-05]
29976
[8.6494118e-01 6.0239919e-02 5.9018475e-01 ... 1.7014621e-05 3.4344492e-05
 9.6308398e-01]
29977
[8.8222927e-01 8.9123344e-01 6.4748949e-01 ... 1.2116350e-04 2.9252964e-04
 9.0439516e-01]
29978
[9.7186971e-01 8.9720160e-01 4.7138527e-01 ... 3.1995660e-05 2.6829905e-05
 7.3822133e-02]
29979
[9.4386268e-01 8.9699388e-01 9.0103501e-01 ... 8.5074134e-05 2.5630635e-04
 9.3104374e-01]
29980
[8.6465758e-01 8.8192207e-01 6.1337417e-01 ... 1.9840694e-04 4.9466384e-04
 4.0771009e-04]
29981
[8.8197327e-01 4.6490175e-01 8.5976474e-02 ... 2.7222375e-05 8.5300286e-05
 7.0491827e-01]
29982
[8.5899389e-01 3.7412125e-01 4.6963636e-02 ... 1.5954427e-05 3.4052278e-05
 9.0773362e-01]
29983
[8.2709438e-01 2.8721964e-01 4.2985447e-02 ... 1.1146901e-05 2.1615902e-05
 7.2555685e-01]
29984
[9.0920991e-01 5.5111825e-01 1.6992432e-01 ... 5.6513993e-05 2.2362756e-04
 9.1853034e-01]
29985
[8.5533792e-01 9.0754706e-01 3.9092103e-01 ... 1.6699256e-05 4.3713189e-05
 5.4556542e-05]
29986
[8.67597

 5.5393094e-01]
30068
[8.6167955e-01 8.3180773e-01 8.4929544e-01 ... 4.6379774e-04 3.7080315e-03
 5.5891222e-01]
30069
[8.8526553e-01 4.5411360e-01 8.4369145e-02 ... 2.5341807e-05 7.3510470e-05
 9.1043083e-05]
30070
[8.5161978e-01 8.6559755e-01 4.5483020e-01 ... 5.4864131e-04 2.5715749e-03
 6.5016544e-01]
30071
[9.2277604e-01 8.7260640e-01 8.8424402e-01 ... 1.4140974e-04 6.8682665e-04
 8.4733313e-01]
30072
[6.4096719e-01 3.5105881e-01 6.4878577e-01 ... 2.3428031e-04 6.4600748e-04
 7.2237211e-01]
30073
[4.6177977e-01 8.2506686e-01 5.5902636e-01 ... 3.4820801e-04 2.0389035e-03
 8.2866800e-01]
30074
[8.9986140e-01 5.3589773e-01 1.1711301e-01 ... 4.8741720e-05 5.3040978e-05
 7.5899434e-01]
30075
[8.7903243e-01 8.5077792e-01 8.6503398e-01 ... 4.5420896e-04 3.3859676e-03
 9.0377170e-01]
30076
[9.5999134e-01 8.7889665e-01 9.0275735e-01 ... 2.4579575e-05 6.0256156e-05
 9.8315436e-01]
30077
[7.8602868e-01 9.0971369e-01 2.2684170e-01 ... 9.1092015e-06 4.3535115e-06
 3.6134638e-02]
30078
[8.66879

 8.8501877e-01]
30157
[9.4763917e-01 8.9577663e-01 9.0499276e-01 ... 8.2164435e-05 1.8175381e-04
 8.5348898e-01]
30158
[8.9417160e-01 9.0326536e-01 2.4748982e-01 ... 6.6902110e-05 3.6049241e-04
 5.6452906e-01]
30159
[8.1993425e-01 9.2137772e-01 2.6907715e-01 ... 9.1220154e-06 1.5521648e-05
 9.0459859e-01]
30160
[7.6613832e-01 9.0708804e-01 2.6214382e-01 ... 1.0661852e-05 1.8260025e-05
 9.8426801e-01]
30161
[9.3284804e-01 8.8924998e-01 8.9865017e-01 ... 6.8070396e-05 3.8413954e-04
 8.1199157e-01]
30162
[9.4663310e-01 9.0294224e-01 9.0649289e-01 ... 6.0792518e-05 2.3894849e-04
 9.2724538e-01]
30163
[8.5408896e-01 9.1381085e-01 4.0340662e-01 ... 1.9600362e-05 1.3541140e-05
 9.4555271e-01]
30164
[9.6750534e-01 8.9716756e-01 9.1959339e-01 ... 3.4518220e-05 3.5385157e-05
 9.5965892e-01]
30165
[9.5452684e-01 8.7050223e-01 9.0422684e-01 ... 6.1041326e-05 1.0369153e-04
 6.8600470e-01]
30166
[9.6141934e-01 7.1636534e-01 1.7607351e-01 ... 6.7187184e-06 8.1973858e-06
 9.5855749e-01]
30167
[8.96989

30243
[9.6243083e-01 7.8410935e-01 9.1645861e-01 ... 1.0349170e-05 1.6756798e-05
 2.0753618e-05]
30244
[9.6934491e-01 8.7352264e-01 9.0642333e-01 ... 2.3802533e-05 4.0148065e-05
 2.8465825e-01]
30245
[9.6141607e-01 8.7548661e-01 9.1178638e-01 ... 4.1041476e-05 6.3051091e-05
 9.6654949e-05]
30246
[9.7241497e-01 8.5094428e-01 9.2134178e-01 ... 1.4580571e-05 2.1339863e-05
 9.6593237e-01]
30247
[6.2395406e-01 2.5890681e-01 6.3828248e-01 ... 1.1069348e-04 2.5951010e-04
 6.4037472e-01]
30248
[9.6179694e-01 8.0292952e-01 9.1528082e-01 ... 1.0450512e-05 2.1455708e-05
 2.4042349e-05]
30249
[9.3692803e-01 8.8039684e-01 8.9131731e-01 ... 7.9185978e-05 5.1671953e-04
 1.0089174e-04]
30250
[8.0847150e-01 2.6156253e-01 3.6040705e-02 ... 9.6591793e-06 1.6404607e-05
 2.0215666e-05]
30251
[8.9846331e-01 9.1046917e-01 5.4081583e-01 ... 4.5946734e-05 1.5003084e-04
 9.3980306e-01]
30252
[9.4124860e-01 8.9374208e-01 8.9996594e-01 ... 6.4211563e-05 2.1634161e-04
 8.6617067e-02]
30253
[9.6143919e-01 8.8221276

30333
[8.9646149e-01 9.1326040e-01 5.1617491e-01 ... 4.2149306e-05 3.9890154e-05
 5.3748596e-01]
30334
[8.81822586e-01 6.67228878e-01 3.19070667e-01 ... 1.14376766e-04
 6.29806018e-04 7.71152377e-01]
30335
[8.5950708e-01 8.1637424e-01 8.3506125e-01 ... 4.6880980e-04 3.9429902e-03
 3.6135900e-01]
30336
[9.5845962e-01 8.9557928e-01 5.0850898e-01 ... 3.8639137e-05 1.3724100e-04
 4.3056294e-01]
30337
[9.0860444e-01 6.3285333e-01 2.5746301e-01 ... 9.6634576e-05 1.4865623e-04
 1.8575687e-04]
30338
[8.7428081e-01 9.1675472e-01 4.3924090e-01 ... 2.4393858e-05 1.6049882e-05
 6.3678759e-01]
30339
[9.5909268e-01 9.1038460e-01 9.1593921e-01 ... 8.7382177e-05 9.5673488e-05
 1.7669442e-04]
30340
[6.8053830e-01 3.8146585e-01 8.0514187e-01 ... 1.8604190e-04 1.5066428e-03
 2.8341249e-01]
30341
[9.1876566e-01 8.8506073e-01 8.9224690e-01 ... 1.4766645e-04 7.4351649e-04
 6.8206245e-01]
30342
[9.3426347e-01 8.8524985e-01 8.9456666e-01 ... 1.1725093e-04 8.1915583e-04
 7.9841119e-01]
30343
[9.5915365e-01 8.7

# Save Model

In [20]:
torch.save(model.state_dict(), "recommender_model.pt")

# Functions For Testing

In [6]:
def create_user_vector(user_id, ratings_completed, num_anime):
    #user_id: id of user
    #ratings_completed: pandas dataframe of anime reviews with completed watching status and rating != 0
    #num_anime: total number of animes in the dataset
    user_rating = ratings_completed.loc[ratings_completed['user_id']==user_id]
    user_anime = user_rating['anime_id']
    user_vector = np.zeros(num_anime)
    for anime in user_anime:
        anime_id = anime_df[anime_df['MAL_ID'] == anime].index[0]
        user_vector[anime_id] = 1
    return user_vector

def create_user_vector_dict(ratings_completed, num_anime):
    user_list = ratings_completed['user_id'].unique()
    user_vector_dict = {}
    for user in user_list:
        user_vector = create_user_vector(user, ratings_completed, num_anime)
        user_vector_dict[user] = user_vector
    return user_vector_dict

In [60]:
def find_closest_user(user_anime, user_set_dict):
    #user_anime: list of anime Ids active user interacted with
    #user_set_dict: dict of userID to their set of animes watched

    active_user_set = set(user_anime)

    closest_user = 0
    closest_distance = float('inf')
    for user in user_set_dict.keys():
        distance = len(active_user_set.symmetric_difference(user_set_dict[user]))
        if distance < closest_distance:
            closest_user = user
            closest_distance = distance
    return closest_user, closest_distance
def create_user_set_dict(ratings_completed):
    #ratings_completed: pandas dataframe of anime reviews with completed watching status and rating != 0
    #num_anime: total number of animes in the dataset
    #anime_df: pandas dataframe of all animes and its features
    user_set_dict = {}
    for review in ratings_completed.itertuples():
        if review[1] in user_set_dict:
            user_set_dict[review[1]].add(review[2])
        else:
            user_set_dict[review[1]] = {review[2]}
    return user_set_dict

In [23]:
def recommend_anime(model, user_id, all_animeIds, anime_df, user_watched = set(), num_recommend = 10):
    #model: NCF model used for recommendation
    #user_id: integer value representing a user
    #all_animeIds: set of all anime Ids
    #user_watched: set of all anime watched by active user
    #num_recommend: number of animes returned by the function
    non_interacted = set(all_animeIds)-set(user_watched)
    non_interacted = list(non_interacted)
    
    non_interacted = list(np.random.choice(non_interacted, 200))
    
    predicted_labels = np.squeeze(model(torch.tensor([user_id]*len(non_interacted)), 
                                        torch.tensor(non_interacted)).detach().numpy())
    recommended_anime = [non_interacted[x] for x in np.argsort(predicted_labels)[::-1][0:num_recommend].tolist()]
    recommended_anime = print_anime(recommended_anime, anime_df)
    return recommended_anime
def print_anime(anime_list, anime_df):
    anime_name_list = list()
    for anime in anime_list:
        anime_name_list.append(anime_df.loc[anime_df['MAL_ID'] == anime]['Name'].item())
    print(anime_name_list)
    return(anime_name_list)

In [61]:
user_set_dict = create_user_set_dict(ratings_completed)

# Save User Interation Dictionary

In [2]:
import pickle
#Load All Anime Ids
with open('all_animeIds.pickle', 'rb') as handle:
    all_animeIds = pickle.load(handle)
#Load User Dictionary
with open('user_set_dict.pickle', 'rb') as handle:
    user_set_dict = pickle.load(handle)
#Load Anime Genres
with open('anime_genres.pickle', 'rb') as handle:
    anime_genres = pickle.load(handle)

In [3]:
#save user_set_dict in a pickle file
with open('user_set_dict.pickle', 'wb') as handle:
    pickle.dump(user_set_dict, handle, protocol= 3)

# Testing (with hypothetical active user)

In [1]:
from classes import *
import torch
from torch.utils.data import Dataset
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import pickle
np.random.seed(10)
ratings = pd.read_csv('animelist.csv')
anime_df = pd.read_csv('anime.csv')
ratings_current = ratings.loc[ratings['watching_status']==1]
rand_userIds = np.random.choice(ratings_current['user_id'].unique(), 
                                size=int(len(ratings_current['user_id'].unique())*0.5), 
                                replace=False)
ratings_target = ratings_current.loc[ratings_current['user_id'].isin(rand_userIds)]

ratings_completed = pd.read_csv('rating_complete.csv')
ratings_completed = ratings_completed.loc[ratings_completed['user_id'].isin(rand_userIds)]
ratings_completed.loc[:, 'rating'] = 1

#Load Model
num_users = ratings['user_id'].max()+1
num_items = ratings['anime_id'].max()+1
all_animeIds = ratings['anime_id'].unique()
model = NCF(num_users, num_items, ratings_completed, all_animeIds)
model.load_state_dict(torch.load("recommender_model.pt"))
model.eval()

#Load User Dictionary
with open('user_set_dict.pickle', 'rb') as handle:
    user_set_dict = pickle.load(handle)

In [28]:
user_watched = ratings_completed[ratings_completed['user_id'] == 0]['anime_id'].values.tolist()
recAnime = recommend_anime(model, 0, all_animeIds, anime_df, user_watched, 15)

['Seitokai Yakuindomo', 'Psycho-Pass 2', 'Shokugeki no Souma: San no Sara', 'Cardcaptor Sakura', '3-gatsu no Lion 2nd Season', 'Itai no wa Iya nano de Bougyoryoku ni Kyokufuri Shitai to Omoimasu.', 'Watashi ga Motete Dousunda', 'Code Geass: Boukoku no Akito 2 - Hikisakareshi Yokuryuu', "Dog Days'", 'Kyoukai no Kanata: Shinonome', 'K-On!: Ura-On!', 'Fractale', 'Saraiya Goyou', 'Medarot', 'Gun x Sword']


In [26]:
recAnime

['Orange',
 'Vampire Knight: Guilty',
 '07-Ghost',
 'Dragon Ball Z Movie 14: Kami to Kami',
 'Rurouni Kenshin: Meiji Kenkaku Romantan',
 'Naruto: Shippuuden Movie 2 - Kizuna',
 'Gintama.: Shirogane no Tamashii-hen',
 'Shouwa Genroku Rakugo Shinjuu',
 'Seishun Buta Yarou wa Yumemiru Shoujo no Yume wo Minai',
 'Joker Game',
 'Okusama ga Seitokaichou!+!',
 'Tensei shitara Slime Datta Ken: Kanwa - Veldora Nikki',
 'Mairimashita! Iruma-kun',
 'Akagami no Shirayuki-hime: Nandemonai Takaramono, Kono Page',
 'Honzuki no Gekokujou: Shisho ni Naru Tame ni wa Shudan wo Erandeiraremasen 2nd Season']

In [136]:
#shounen_vector = [20, 1735, 269, 6702, 22043, 22319, 36511, 5114, 2001, 249, 136, 137, 138, 139, 223, 813, 3588, 21, 288, 392, 482, 14513, 18115, 235, 24833, 30654, 665, 666, 3603, 14719, 20899, 26055, 9919, 1604]
#shounen_closest_user = find_closest_user(shounen_vector, user_set_dict, 17562)[0]
romcom_vector = [4224, 59, 120, 853, 13659, 18857, 14813, 23847, 39547, 18897, 27787, 17895, 2167, 4181, 34280, 8525, 10080, 15117, 16706, 28677, 15809, 48413, 13759, 7054, 9989, 28851, 23273, 37999, 40591, 35968]
romcom_closest_user = find_closest_user(romcom_vector, user_set_dict)[0]
user_watched = ratings_completed[ratings_completed['user_id'] == romcom_closest_user]['anime_id'].values.tolist()
recommend_anime(model, romcom_closest_user, all_animeIds, anime_df, user_watched, 15)

['Higashi no Eden', 'Boku dake ga Inai Machi', 'Hataraku Maou-sama!', 'Cowboy Bebop: Tengoku no Tobira', 'Kuroko no Basket', 'Hunter x Hunter (2011)', 'Made in Abyss', 'Naruto', 'Koukyoushihen Eureka Seven', 'Black Lagoon: The Second Barrage', 'Gekkan Shoujo Nozaki-kun', 'Natsume Yuujinchou', 'Kono Subarashii Sekai ni Shukufuku wo! 2', 'Mirai Nikki', 'Nichijou']


['Higashi no Eden',
 'Boku dake ga Inai Machi',
 'Hataraku Maou-sama!',
 'Cowboy Bebop: Tengoku no Tobira',
 'Kuroko no Basket',
 'Hunter x Hunter (2011)',
 'Made in Abyss',
 'Naruto',
 'Koukyoushihen Eureka Seven',
 'Black Lagoon: The Second Barrage',
 'Gekkan Shoujo Nozaki-kun',
 'Natsume Yuujinchou',
 'Kono Subarashii Sekai ni Shukufuku wo! 2',
 'Mirai Nikki',
 'Nichijou']

In [37]:
romcom_closest_user

53711

In [116]:
ratings_completed[ratings_completed['user_id'] == 7]

,user_id,anime_id,rating


In [121]:
user = 19
user_watched = ratings_completed[ratings_completed['user_id'] == user]['anime_id'].values.tolist()
recommend_anime(model, user, all_animeIds, anime_df, user_watched)

['Neon Genesis Evangelion', 'JoJo no Kimyou na Bouken (TV)', 'Koutetsujou no Kabaneri', 'Naruto: Shippuuden Movie 1', 'Naruto: Shippuuden Movie 1', 'Orange', 'Kämpfer', '91 Days', 'Oda Nobuna no Yabou', 'One Piece Film: Strong World']


['Neon Genesis Evangelion',
 'JoJo no Kimyou na Bouken (TV)',
 'Koutetsujou no Kabaneri',
 'Naruto: Shippuuden Movie 1',
 'Naruto: Shippuuden Movie 1',
 'Orange',
 'Kämpfer',
 '91 Days',
 'Oda Nobuna no Yabou',
 'One Piece Film: Strong World']

In [118]:
ratings_completed[(ratings_completed['anime_id'] == 16498)]

,user_id,anime_id,rating
130,1,16498,1
2648,19,16498,1
4042,29,16498,1
4858,40,16498,1
5958,44,16498,1
...,...,...,...
57630091,353383,16498,1
57630937,353388,16498,1
57631312,353392,16498,1
57631971,353395,16498,1


In [144]:
ratings[(ratings['user_id'] == 53711)]# & (ratings['rating'] != 0)]

,user_id,anime_id,rating,watching_status,watched_episodes
16573025,53711,39783,0,1,7
16573026,53711,45782,0,1,3
16573027,53711,42897,0,1,7
16573028,53711,40530,0,1,7
16573029,53711,40750,0,1,6
...,...,...,...,...,...
16576060,53711,42962,0,6,0
16576061,53711,44940,0,6,0
16576062,53711,42506,0,6,0
16576063,53711,38349,0,6,0


In [145]:
anime_df[anime_df['MAL_ID'] == 1535]

,MAL_ID,Name,Score,Genders,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
1393,1535,Death Note,8.63,"Mystery, Police, Psychological, Supernatural, ...",Death Note,デスノート,TV,37,"Oct 4, 2006 to Jun 27, 2007",Fall 2006,...,557406.0,535252.0,415890.0,201522.0,68577.0,28048.0,10462.0,3692.0,2256.0,3586.0


In [54]:
genres = []
for anime in anime_df['Genders']:
    genres.extend(anime.split(", "))
    
genres = list(dict.fromkeys(genres)) 

In [78]:
genres

['Action',
 'Adventure',
 'Comedy',
 'Drama',
 'Sci-Fi',
 'Space',
 'Mystery',
 'Shounen',
 'Police',
 'Supernatural',
 'Magic',
 'Fantasy',
 'Sports',
 'Josei',
 'Romance',
 'Slice of Life',
 'Cars',
 'Seinen',
 'Horror',
 'Psychological',
 'Thriller',
 'Super Power',
 'Martial Arts',
 'School',
 'Ecchi',
 'Vampire',
 'Military',
 'Historical',
 'Dementia',
 'Mecha',
 'Demons',
 'Samurai',
 'Game',
 'Shoujo',
 'Harem',
 'Music',
 'Shoujo Ai',
 'Shounen Ai',
 'Kids',
 'Hentai',
 'Parody',
 'Yuri',
 'Yaoi']

In [5]:
#save anime genres
with open('anime_genres.pickle', 'wb') as handle:
    pickle.dump(anime_genres, handle, protocol=3)

In [63]:
#save variable ratings_completed
with open('ratings_completed_used.pickle', 'wb') as handle:
    pickle.dump(ratings_completed, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [75]:
anime_df[anime_df['Genders'] == 'Unknown']

,MAL_ID,Name,Score,Genders,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
9788,28487,Ikite Iru,Unknown,Unknown,Unknown,生きている,OVA,1,1996,Unknown,...,4.0,Unknown,2.0,1.0,2.0,8.0,Unknown,Unknown,Unknown,3.0
9838,28653,Maze,Unknown,Unknown,Unknown,MAZE,Movie,1,2012,Unknown,...,7.0,Unknown,4.0,11.0,19.0,18.0,15.0,5.0,11.0,10.0
9839,28655,PiKA PiKA,5.12,Unknown,Unknown,PiKA PiKA,Movie,1,2006,Unknown,...,4.0,2.0,5.0,27.0,45.0,55.0,47.0,18.0,23.0,35.0
10090,29655,Chanda Gou,Unknown,Unknown,M.S. Chanda,チャンダ号,Movie,1,1964,Unknown,...,5.0,2.0,1.0,7.0,10.0,19.0,14.0,7.0,13.0,11.0
10145,29765,Metropolis (2009),5.93,Unknown,Unknown,METROPOLIS,Movie,1,2009,Unknown,...,18.0,11.0,31.0,33.0,31.0,39.0,12.0,12.0,9.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15964,40090,Sinbi Apateu: Ghost Ball X-ui Tansaeng,Unknown,Unknown,Unknown,신비아파트:고스트볼X의 탄생,TV,23,"Nov 9, 2017 to Jan 24, 2019",Fall 2017,...,5.0,4.0,2.0,4.0,4.0,3.0,3.0,2.0,1.0,2.0
16324,40717,Kaiju Decode,Unknown,Unknown,Unknown,KAIJU DECODE,Unknown,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
17304,43762,Hula Fulla Dance,Unknown,Unknown,Hula Fulla Dance,フラ・フラダンス,Movie,1,2021,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
17326,44041,SD Gundam World Heroes,Unknown,Unknown,SD GUNDAM WORLD HEROES,SDガンダムワールド ヒーローズ,TV,Unknown,"Apr, 2021 to ?",Spring 2021,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown


In [3]:
ratings = pd.read_csv('animelist.csv')
all_animeIds = ratings['anime_id'].unique()

In [6]:
with open('all_animeIds.pickle', 'wb') as handle:
    pickle.dump(all_animeIds, handle, protocol=3)